<a href="https://colab.research.google.com/github/fbbgarcia/fbgarcia-lab7/blob/main/CSC_365_Lab_7_Database_Connectivity_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [415]:
pip install mysql-connector-python

In [416]:
import pandas as pd
import getpass
import mysql.connector
from IPython.display import display
from datetime import datetime
from datetime import datetime, timedelta

In [417]:
db_password = getpass.getpass()

··········


In [418]:
conn = mysql.connector.connect(user='fbgarcia', password=db_password,
                               host='mysql.labthreesixfive.com',
                               database='fbgarcia')
cursor = conn.cursor()

# Implementation

In [419]:
def main():
  print("Welcome \n\nMain Menu: \n1. Rooms and Rates \n2. Make a Reservation\n3. Cancel a Reservation\n4. Reservation Details \n5. Revenue Info \n")

  main_input = input("Please select an option: ")

  if main_input == "1":
    fr1()

  if main_input == "2":
    fr2()

  if main_input == "3":
    fr3()

  if main_input == "4":
    print("Please fill out the form above to search for a reservation.")

  if main_input == "5":
    fr5()

In [420]:
def fr1():
  fr1_df = pd.read_sql("""
  WITH score AS (
  SELECT RoomCode, ROUND(SUM(
          DATEDIFF(
              LEAST(CheckOut, CURDATE()),
              GREATEST(CheckIn, DATE_SUB(CURDATE(), INTERVAL 180 DAY))))/180, 2)
          AS PopularityScore
  FROM lab7_rooms JOIN lab7_reservations ON RoomCode = Room
  WHERE CheckIn BETWEEN DATE_SUB(CURDATE(), INTERVAL 180 DAY) AND CURDATE()
      OR CheckOut BETWEEN DATE_SUB(CURDATE(), INTERVAL 180 DAY) AND CURDATE()
  GROUP BY RoomCode
  ),

  next AS (
  SELECT Room, DATE_ADD(
              CURDATE(),
              INTERVAL GREATEST(MIN(DATEDIFF(CheckOut, CURDATE())), 0) DAY)
              AS NextAvailableCheckIn
  FROM lab7_reservations
  WHERE CODE NOT IN (
  SELECT rOut.CODE
  FROM lab7_reservations AS rOut JOIN lab7_reservations AS rIn
      ON rOut.Room = rIn.Room
      AND rOut.CheckOut = rIn.CheckIn
  )
  GROUP BY Room
  ),

  length AS (SELECT RoomCode, DATEDIFF(CheckOut, CheckIn) AS LastResvLength
  FROM lab7_rooms JOIN lab7_reservations ON RoomCode = Room
  WHERE (RoomCode, CheckOut) In (
      SELECT RoomCode, MAX(CheckOut)
      FROM lab7_rooms JOIN lab7_reservations ON RoomCode = Room
      GROUP BY RoomCode)
  )

  SELECT lab7_rooms.*, PopularityScore, NextAvailableCheckIn, LastResvLength
  FROM lab7_rooms JOIN score ON lab7_rooms.roomCode = score.RoomCode
      JOIN next ON score.RoomCode = next.Room
      JOIN length ON score.RoomCode = length.RoomCode
  ORDER BY PopularityScore DESC
  """, conn)

  fr1_df = fr1_df.rename(columns = {"RoomCode": "Room Code",
                            "RoomName": "Room Name",
                            "Beds": "Number of Beds",
                            "bedType": "Bed Type",
                            "maxOcc": "Maximum Occupancy",
                            "basePrice": "Base Price",
                            "decor": "Decor",
                            "PopularityScore": "Popularity Score",
                            "NextAvailableCheckIn": "Next Available Check-In",
                            "LastResvLength": "Length of Last Reservation (Days)"})
  fr1_df.index += 1
  display(fr1_df)

In [421]:
def fr2():
  firstName = input('Please enter your first name: ')
  lastName = input('Please enter your last name: ')
  roomCode = input('Please enter your desired room code, or "Any" to indicate no room preferrence: ')
  bedType = input('Please enter your desired bed type, or "Any" to indicade no bed type preferrence: ')
  start = input('Please enter your desired check-in date (YYYY-MM-DD): ')
  end = input('Please enter your desired check-out date (YYYY-MM-DD): ')
  children = input('Please enter the number of children that will be staying in the room: ')
  adults = input('Please enter the number of adults that will be staying in the room: ')

  global search_parameters
  global room_codes
  global room_code_placeholders
  global bed_types
  global bed_types_placeholders

  search_parameters = [roomCode, bedType, children, adults, start, end]



  # check if desired occupancy exceeds maximum occupancy available
  cursor.execute("""
  SELECT MAX(maxOcc)
  FROM lab7_rooms
  """)
  result = cursor.fetchall()

  if int(search_parameters[2])+int(search_parameters[3]) > int(result[0][0]):
    print("No rooms satisfying your requested occupancy are available.")



  else:
    # check for roomcode preference
    if search_parameters[0] == "Any":
      search_parameters

      cursor.execute("""
      SELECT DISTINCT RoomCode
      FROM lab7_rooms
      """)
      result = cursor.fetchall()
      room_codes = [row[0] for row in result]
      search_parameters = room_codes + search_parameters[1:]

    else:
      room_codes = [search_parameters[0]]

    room_code_placeholders = ", ".join(["%s"] * len(room_codes))



    # check for bed type preference
    if search_parameters[len(room_codes)] == "Any":
      cursor.execute("""
      SELECT DISTINCT bedType
      FROM lab7_rooms
      """)
      result = cursor.fetchall()
      bed_types = [row[0] for row in result]
      search_parameters = search_parameters[:len(room_codes)] + bed_types + search_parameters[len(room_codes)+1:]

    else:
      bed_types = [search_parameters[len(room_codes)]]

    bed_types_placeholders = ", ".join(["%s"] * len(bed_types))




  exact_matches = search1()
  if len(exact_matches) > 0:
    fr2_df = pd.DataFrame(set(exact_matches))
    fr2_df = fr2_df.rename(columns = {0: "Room Code",
                              1: "Room Name",
                              2: "Number of Beds",
                              3: "Bed Type",
                              4: "Maximum Occupancy",
                              5: "Base Price",
                              6: "Decor"})
    fr2_df.index += 1
    print("\nHere are your matching results:")
    display(fr2_df)
    request_choice = input("\nWould you like to request a reservation? (Yes/No): ")

    if request_choice == "Yes":
      book_choice = input("Please select the number of the room you would like to book from the list above: ")
      room = fr2_df.loc[int(book_choice)]

      cursor.execute("""
      SELECT MAX(Code)
      FROM lab7_reservations
      """)
      result = cursor.fetchall()
      new_code = result[0][0]+1

      base_rate = float(room["Base Price"])
      start_dt = datetime.strptime(start, "%Y-%m-%d").date()
      end_dt = datetime.strptime(end, "%Y-%m-%d").date()
      weekdays_count = 0
      weekends_count = 0

      current_date = start_dt
      while current_date < end_dt:
          if current_date.weekday() < 5:
              weekdays_count += 1
          else:
              weekends_count += 1
          current_date += timedelta(days=1)
      cost = (base_rate*weekdays_count) + (base_rate*1.10*weekends_count)
      cost = round(cost, 2)
      rate = round(cost/(weekdays_count+weekends_count), 2)

      info = [new_code, room["Room Code"], start, end, rate, lastName, firstName, int(adults), int(children)]
      confirmation_info = [new_code, firstName, lastName, start, end, adults, children, cost]
      columns = ["Reservation Code", "First Name", "Last Name", "Check-In", "Check-Out", "Adults", "Children", "Total Cost"]

      confirmation = pd.DataFrame([confirmation_info], columns = columns)

      cursor.execute("""
      INSERT INTO lab7_reservations
      VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)
      """, info)

      print("\nYour room has been reserved. Here is your booking confirmation:")
      display(confirmation)
      print("\nReturning to the Main Menu...\n")
      main()


    elif request_choice == "No":
      print("Returning to the Main Menu...\n")
      main()



  else:
    fr2_df = pd.DataFrame(set(search2()))
    fr2_df = fr2_df.rename(columns = {0: "Room Code",
                               1: "Room Name",
                               2: "Number of Beds",
                               3: "Bed Type",
                               4: "Maximum Occupancy",
                               5: "Base Price",
                               6: "Decor"})
    fr2_df = fr2_df.head(5)
    fr2_df.index += 1
    if len(fr2_df) > 0:
      print("Sorry, no rooms matching your requests are available. \nHere are similar rooms available during your selected dates:")
      display(fr2_df)
      request_choice = input("Would you like to request a reservation? (Yes/No): ")

      if request_choice == "Yes":
        book_choice = input("Please select the number of the room you would like to book from the list above: ")
        room = fr2_df.loc[int(book_choice)]

        cursor.execute("""
        SELECT MAX(Code)
        FROM lab7_reservations
        """)
        result = cursor.fetchall()
        new_code = result[0][0]+1

        base_rate = float(room["Base Price"])
        start_dt = datetime.strptime(start, "%Y-%m-%d").date()
        end_dt = datetime.strptime(end, "%Y-%m-%d").date()
        weekdays_count = 0
        weekends_count = 0

        current_date = start_dt
        while current_date < end_dt:
            if current_date.weekday() < 5:
                weekdays_count += 1
            else:
                weekends_count += 1
            current_date += timedelta(days=1)
        cost = (base_rate*weekdays_count) + (base_rate*1.10*weekends_count)
        cost = round(cost, 2)
        rate = round(cost/(weekdays_count+weekends_count), 2)

        info = [new_code, room["Room Code"], start, end, rate, lastName, firstName, int(adults), int(children)]
        confirmation_info = [new_code, firstName, lastName, start, end, adults, children, cost]
        columns = ["Reservation Code", "First Name", "Last Name", "Check-In", "Check-Out", "Adults", "Children", "Total Cost"]

        confirmation = pd.DataFrame([confirmation_info], columns = columns)

        cursor.execute("""
        INSERT INTO lab7_reservations
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, info)

        print("\nYour room has been reserved. Here is your booking confirmation:")
        display(confirmation)
        print("\nReturning to the Main Menu...\n")
        main()


      elif request_choice == "No":
        print("Returning to the Main Menu...\n")
        main()

    else:
      print("\nSorry, no rooms are available during your selected dates.")
      print("Returning to the Main Menu...\n")
      main()



# search for exact matches
def search1():
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE RoomCode IN ({room_code_placeholders})
          AND bedType IN ({bed_types_placeholders})
          AND maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, search_parameters)
  result = cursor.fetchall()
  return result



# search for similar matches
def search2():
  cursor.execute(f"""
  SELECT decor
  FROM lab7_rooms
  WHERE RoomCode IN ({room_code_placeholders})
  """, room_codes)
  result = cursor.fetchall()
  decors = [row[0] for row in result]
  decors_placeholders = ", ".join(["%s"] * len(decors))

  # different room code (similar decor)
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE decor IN ({decors_placeholders})
          AND bedType IN ({bed_types_placeholders})
          AND maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, decors + bed_types + search_parameters[len(room_codes)+len(bed_types):])
  result = cursor.fetchall()
  similar_matches = result

  # different decor
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE bedType IN ({bed_types_placeholders})
          AND maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, bed_types + search_parameters[len(room_codes)+len(bed_types):])
  result = cursor.fetchall()
  similar_matches = similar_matches + result

  # different bed type
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, search_parameters[len(room_codes)+len(bed_types):])
  result = cursor.fetchall()
  similar_matches = similar_matches + result



  return similar_matches



In [422]:
# search for exact matches
def search1():
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE RoomCode IN ({room_code_placeholders})
          AND bedType IN ({bed_types_placeholders})
          AND maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, search_parameters)
  result = cursor.fetchall()
  return result



# search for similar matches
def search2():
  cursor.execute(f"""
  SELECT decor
  FROM lab7_rooms
  WHERE RoomCode IN ({room_code_placeholders})
  """, room_codes)
  result = cursor.fetchall()
  decors = [row[0] for row in result]
  decors_placeholders = ", ".join(["%s"] * len(decors))

  # different room code (similar decor)
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE decor IN ({decors_placeholders})
          AND bedType IN ({bed_types_placeholders})
          AND maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, decors + bed_types + search_parameters[len(room_codes)+len(bed_types):])
  result = cursor.fetchall()
  similar_matches = result

  # different decor
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE bedType IN ({bed_types_placeholders})
          AND maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, bed_types + search_parameters[len(room_codes)+len(bed_types):])
  result = cursor.fetchall()
  similar_matches = similar_matches + result

  # different bed type
  cursor.execute(f"""
  SELECT *
  FROM lab7_rooms
  WHERE RoomCode IN (
      SELECT RoomCode
      FROM lab7_rooms
      WHERE maxOcc >= (%s + %s)
  ) AND RoomCode NOT IN (
      SELECT Room
      FROM lab7_reservations
      WHERE %s BETWEEN CheckIn AND DATE_SUB(CheckOut, INTERVAL 1 DAY)
          OR %s BETWEEN DATE_SUB(CheckIn, INTERVAL 1 DAY) AND CheckOut
  )
  """, search_parameters[len(room_codes)+len(bed_types):])
  result = cursor.fetchall()
  similar_matches = similar_matches + result



  return similar_matches


In [423]:
def fr3():
  code = input("\nPlease enter the code of the reservation you'd like to cancel or 'Back' to retun to the Main Menu: ")

  if code == "Back":
    main()

  else:
    confirmation = input("\nAre you sure you want to cancel reservation " + code +"? (Yes/No): ")

    if confirmation == "Yes":
      cursor.execute("""
      DELETE FROM lab7_reservations
      WHERE CODE = 10105
      """)
      print("\nReservation "+ code + " has been canceled.")
      print("Returning to the Main Menu...\n")
      main()

    elif confirmation == "No":
      fr3()


In [424]:
def fr5():
    cursor.execute("""
    WITH revenue AS (
      SELECT
          lab7_rooms.RoomName,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-01-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-01-01'))) + 1) * lab7_reservations.Rate), 0) AS January,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-02-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-02-01'))) + 1) * lab7_reservations.Rate), 0) AS February,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-03-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-03-01'))) + 1) * lab7_reservations.Rate), 0) AS March,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-04-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-04-01'))) + 1) * lab7_reservations.Rate), 0) AS April,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-05-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-05-01'))) + 1) * lab7_reservations.Rate), 0) AS May,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-06-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-06-01'))) + 1) * lab7_reservations.Rate), 0) AS June,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-07-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-07-01'))) + 1) * lab7_reservations.Rate), 0) AS July,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-08-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-08-01'))) + 1) * lab7_reservations.Rate), 0) AS August,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-09-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-09-01'))) + 1) * lab7_reservations.Rate), 0) AS September,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-10-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-10-01'))) + 1) * lab7_reservations.Rate), 0) AS October,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-11-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-11-01'))) + 1) * lab7_reservations.Rate), 0) AS November,
          ROUND(SUM(
              GREATEST(0, DATEDIFF(LEAST(lab7_reservations.Checkout, DATE_ADD(CONCAT(YEAR(CURDATE()), '-12-01'), INTERVAL 1 MONTH)),
              GREATEST(lab7_reservations.CheckIn, CONCAT(YEAR(CURDATE()), '-12-01'))) + 1) * lab7_reservations.Rate), 0) AS December
      FROM
          lab7_rooms
      LEFT JOIN
          lab7_reservations ON lab7_rooms.RoomCode = lab7_reservations.Room
      GROUP BY
          lab7_rooms.RoomName
    )
    SELECT *,
           ROUND((January + February + March + April + May + June + July + August + September + October + November + December), 0) AS Total
    FROM revenue
    ORDER BY RoomName;
    """)
    results = cursor.fetchall()

    cols = ["Room", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "Total"]
    df = pd.DataFrame(results, columns=cols)

    total_row = {
        "Room": "Total",
        'Jan': df['Jan'].sum(),
        'Feb': df['Feb'].sum(),
        'Mar': df['Mar'].sum(),
        'Apr': df['Apr'].sum(),
        'May': df['May'].sum(),
        'Jun': df['Jun'].sum(),
        'Jul': df['Jul'].sum(),
        'Aug': df['Aug'].sum(),
        'Sep': df['Sep'].sum(),
        'Oct': df['Oct'].sum(),
        'Nov': df['Nov'].sum(),
        'Dec': df['Dec'].sum(),
        'Total': df['Total'].sum()
    }

    # Round the totals before appending the total row
    total_row = {key: round(value) if isinstance(value, (int, float)) else value for key, value in total_row.items()}

    # Append the total row to the DataFrame
    df = pd.concat([df, pd.DataFrame([total_row])], ignore_index=True)

    # Adjust the index to start from 1
    df.index += 1

    # Display the DataFrame
    display(df)


# Interface

In [425]:
def fr4():
  FirstName = ""  # @param {"type":"string","placeholder":"First Name"}
  LastName = ""   # @param {"type":"string","placeholder":"Last Name"}
  CheckIn = ""    # @param {"type":"string","placeholder":"YYYY-MM-DD"}
  CheckOut = ""   # @param {"type":"string","placeholder":"YYYY-MM-DD"}
  Room = ""       # @param {"type":"string","placeholder":"XXX"}
  Code = ""       # @param {"type":"string","placeholder":"00000"}

  if (FirstName or LastName or CheckIn or CheckOut or Room or Code):

      query = """
      SELECT lab7_reservations.Code, lab7_reservations.Room, lab7_reservations.CheckIn, lab7_reservations.CheckOut,
            lab7_reservations.Rate, lab7_reservations.LastName, lab7_reservations.FirstName, lab7_rooms.RoomName
      FROM lab7_reservations
      JOIN lab7_rooms ON lab7_reservations.Room = lab7_rooms.RoomCode
      WHERE 1=1
      """

      params = []

      if FirstName:
          query += " AND lab7_reservations.FirstName LIKE %s"
          params.append(f"%{FirstName}%")

      if LastName:
          query += " AND lab7_reservations.LastName LIKE %s"
          params.append(f"%{LastName}%")

      if CheckIn:
          query += " AND lab7_reservations.CheckIn >= %s"
          params.append(CheckIn)

      if CheckOut:
          query += " AND lab7_reservations.CheckOut <= %s"
          params.append(CheckOut)

      if Room:
          query += " AND lab7_reservations.Room LIKE %s"
          params.append(f"%{Room}%")

      if Code:
          query += " AND lab7_reservations.Code LIKE %s"
          params.append(f"%{Code}%")

      cursor.execute(query, params)
      results = cursor.fetchall()

      fr4_df = pd.DataFrame(results)
      fr4_df = fr4_df.rename(columns={
          0: "Reservation Code",
          1: "Room Code",
          2: "Check-In",
          3: "Check-Out",
          4: "Total Cost",
          5: "Last Name",
          6: "First Name",
          7: "Room Name"
      })
      fr4_df.index += 1

      display(fr4_df)


In [426]:
#main()